In [0]:

import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import trim, col   

#Read table from bronze

In [0]:
df = spark.table("workspace.bronze.loc_erp")

#data cleaning

#Triming

In [0]:
for filed in df.schema.fields:
  if isinstance(filed.dataType, StringType):
    df = df.withColumn(filed.name, trim(col(filed.name)))


##Customer ID Cleanup


In [0]:
df = df.withColumn("cid", F.regexp_replace(col("cid"), "-", ""))

## Country Normalization



In [0]:

df = df.withColumn(
    "cntry",
    F.when(col("cntry") == "DE", "Germany")
     .when(col("cntry").isin("US", "USA"), "United States")
     .when((col("cntry") == "") | col("cntry").isNull(), "n/a")
     .otherwise(col("cntry"))
)

##Renaming Columns


In [0]:

RENAME_MAP = {
    "cid": "customer_number",
    "cntry": "country"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

#Write to silver layer

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customer_location")